In [357]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import openpyxl
import geopandas as gpd
import math
import itertools
from scipy import stats
import scipy
import timeit
from functools import reduce
from pandas import ExcelWriter

pd.set_option('display.max_columns', None)
idx = pd.IndexSlice

In [194]:
scipy.version.version

'1.4.1'

In [195]:
input_path = '../../data/input/dhis2/new_system/'
input_path_old = '../../data/input/dhis2/old_system/'

shapes_path = '../../data/shapes/district/districts_17_19.shp'
facility_path = '../../data/input/hospitals/original_data/'

output_path = '../../data/output/sprint3_analysis/'

# Some functions I'll use

In [320]:
shapes.copy().drop('geometry',axis=1).to_csv(output_path+'district_correspondance.csv')

In [196]:
districts = list(pd.read_csv(input_path+'districts.csv')["Districts"])
shapes = gpd.read_file(shapes_path)

# Creating a dict of names to replace district names

def get_district_name_dict (df):
    keys = list(set(df.index).difference(set(shapes['name_19'])))
    values = list(set(shapes['name_19']).difference(set(df.index)))
    district_name = dict(zip(keys, values))
    return district_name 


In [197]:
# build a small fucntion to split the string column name of the data download as pivot 

def split(strng, sep, occ):
    strng = strng.split(sep)
    return sep.join(strng[occ[1]:]), sep.join(strng[:occ[0]]), sep.join(strng[occ[0]:occ[1]])[:3]

In [351]:
# To clean the data downloaded in a pivot format

def get_clean_pivot(df,drop):
    #df['facility']=df['orgunitlevel5']
    #df['id']=df['organisationunitid']
    df['district']=df['orgunitlevel3'].apply(lambda x: x[:-9].upper())
    df['district'].rename(get_district_name_dict(df),inplace=True)
    df.set_index(['district','organisationunitid'],drop=True,inplace=True)
    cols = np.arange(0,9)
    df.drop(df.columns[cols],axis=1,inplace=True)
    cols = df.columns
    new_cols=[]
    for col in cols:
        new_cols.append(split(col,' ',[-2,-1]))
    df.columns=pd.MultiIndex.from_tuples(new_cols)
    if drop != None:
        df.drop(drop,axis=1,inplace=True,level=2)
    return df

In [199]:
# To get year on year deltas over a list of months

def get_delta(df,st_year,end_year,indic,st_month,end_month):
    delta=((df[(end_year,indic,end_month)]
            -df[(st_year,indic,st_month)])
           /df[(st_year,indic,st_month)])
    return delta

In [200]:
# To get month on month deltas

def get_yoy_delta(df,indic,period):
    tot_18 = tot_19= tot_20 = np.zeros(135)
    for month in period:
        tot_18= tot_18+list(df[('2018',indic,month)])
        tot_19= tot_19+list(df[('2019',indic,month)])
        tot_20= tot_20+list(df[('2020',indic,month)])
    delta =  (tot_20-tot_19)/tot_19
    return tot_18,tot_19,tot_20,delta

# Fetch data


## New data


In [390]:
# To update as I go

all_months=['Jan','Feb','Mar','Apr','May']

In [401]:
def fetch_new_data(drop):
    all_months=['Jan','Feb','Mar','Apr','May']
    epi_df = get_clean_pivot(pd.read_csv(input_path+'new_epi_data_by_facility.csv'),drop=drop)
    mnch_df = get_clean_pivot(pd.read_csv(input_path+'new_mnch_data_by_facility.csv'),drop=drop)
    sam_df = get_clean_pivot(pd.read_csv(input_path+'new_sam_data_by_facility.csv'),drop=drop)
    
    df1 = pd.merge(epi_df,mnch_df,left_index=True,right_index=True)
    df2 = pd.merge(df1,sam_df,left_index=True,right_index=True)

    df3 = df2.stack(level=[0,2],dropna=False)
    df4 = df2.stack(level=1,dropna=False)
    
    df3.index.set_names(['district','id','year','month'],inplace=True)
    df4.index.set_names(['district','id','indic'],inplace=True)
    
    columns = [['with_outliers'], ['2020'], all_months]
    df4.columns=pd.MultiIndex.from_product(columns, names=['type','year','month'])
    
    return df3,df4

new_df,new_df1=fetch_new_data(drop='Dec')

C:\Users\Dalberg\anaconda3\envs\icohs\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [402]:
new_df1.index.levels[2]

Index(['105-AN01a. ANC 1st Visit for women',
       '105-AN02. ANC 4th Visit for women', '105-MA01. Admissions',
       '105-MA04a. Deliveries in unit - Total',
       '105-MA04b1. Deliveries in unit -Live births - Total',
       '105-MA04c1. Deliveries in unit - Fresh still birth - Total',
       '105-MA04d1. Deliveries in unit - Macerated still birth - Total',
       '105-MA11. Newborn deaths (0-7 days)',
       '105-NA03a1. Identified malnourished clients(<10) this month - MAM using MUAC',
       '105-NA03b1. Identified malnourished clients(<10) this month - MAM using W/H or L',
       '105-NA03c1. Identified malnourished clients(<10) this month - SAM using MUAC -  Without Oedema',
       '105-NA03d1. Identified malnourished clients(<10) this month - SAM using W/H or L -  Without Oedema',
       '105-NA03e1. Identified malnourished clients(<10) this month - SAM With Oedema',
       '105-NA03g1. Identified malnourished clients(<10) this month - Severely Malnourished',
       '105-PN0

In [ ]:
'105-AN01a. ANC 1st Visit for women',
'105-AN02. ANC 4th Visit for women', 
'105-MA01. Admissions',
'105-MA04b1. Deliveries in unit -Live births - Total',
'105-MA04c1. Deliveries in unit - Fresh still birth - Total',
'105-MA04d1. Deliveries in unit - Macerated still birth - Total',
'105-MA11. Newborn deaths (0-7 days)',

## Old data

For now extremely messy, had to be done bit by bit in random order, so quite some cleaning needed Ill do here, than put all into one nice file

In [360]:
bcg=get_clean_pivot(pd.read_csv(input_path_old+"/epi/EPI - BCG doses given.csv"),drop=None)
dpt1=get_clean_pivot(pd.read_csv(input_path_old+"/epi/EPI - DPT-HepB-HIB 1 doses given.csv"),drop=None)
dpt3=get_clean_pivot(pd.read_csv(input_path_old+"/epi/EPI - DPT-HepB-HIB 3 doses given.csv"),drop=None)
pcv1=get_clean_pivot(pd.read_csv(input_path_old+"/epi/EPI - PCV 1 doses given.csv"),drop=None)
pcv3=get_clean_pivot(pd.read_csv(input_path_old+"/epi/EPI - PCV 3 doses given.csv"),drop=None)
mr1=get_clean_pivot(pd.read_csv(input_path_old+"/epi/EPI - MR 1 doses given.csv"),drop=None)

epi_dfs=[bcg,dpt1,dpt3,pcv1,pcv3,mr1]
old_epi = reduce(lambda  left,right: pd.merge(left,right,left_index=True,right_index=True,how='left'), epi_dfs)

In [443]:
newborn=get_clean_pivot(pd.read_csv(input_path_old+'/mat/admission_newborn.csv'),drop=None)
anc=get_clean_pivot(pd.read_csv(input_path_old+'/mat/ANC1_ANC4.csv'),drop=None)
births=get_clean_pivot(pd.read_csv(input_path_old+'/mat/births.csv'),drop=None)

mat_dfs=[newborn,anc,births]
old_mat = reduce(lambda  left,right: pd.merge(left,right,left_index=True,right_index=True,how='left'),mat_dfs)

In [366]:
lbw=get_clean_pivot(pd.read_csv(input_path_old+'/sam/lbw.csv'),drop=None)
sam=get_clean_pivot(pd.read_csv(input_path_old+'/sam/sam_mam.csv'),drop=None)

sam_dfs=[lbw,sam]
old_sam= reduce(lambda  left,right: pd.merge(left,right,left_index=True,right_index=True,how='left'),sam_dfs)

In [368]:
old_epi.to_csv(input_path_old+'/clean/old_epi_data_by_facility.csv')
old_mat.to_csv(input_path_old+'/clean/old_mat_data_by_facility.csv')
old_sam.to_csv(input_path_old+'/clean/old_sam_data_by_facility.csv')

In [444]:
def fetch_old_data():
    all_months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    df1 = pd.merge(old_epi,old_mat,left_index=True,right_index=True)
    df2 = pd.merge(df1,old_sam,left_index=True,right_index=True)
    
    df3 = df2.stack(level=[0,2],dropna=False)
    df4 = df2.stack(level=1,dropna=False)
    
    df3.index.set_names(['district','id','year','month'],inplace=True)
    df4.index.set_names(['district','id','indic'],inplace=True)
    
    columns = [['with_outliers'], ['2018','2019'], all_months]
    df4.columns=pd.MultiIndex.from_product(columns, names=['type','year','month'])
    return df3,df4

old_df,old_df1=fetch_old_data()

# Merge the two

In [415]:
#Not exactly the samenumber of facilities, but hopefully the ids remained the same 

print(len(old_epi),len(pd.read_csv(input_path+'new_epi_data_by_facility.csv')))

7168 7643


C:\Users\Dalberg\anaconda3\envs\icohs\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Select a subset of indicators to merge on

### Link EPI, Mat, and LBW inidcators together

This is done in Excel manually, after running the small function below

In [442]:
#with ExcelWriter(input_path+'new_old_correspondance.xlsx',mode='a') as writer:
    #pd.Series(old_df1.index.levels[2]).to_excel(writer,sheet_name='old_vars')
    #pd.Series(new_df1.index.levels[2]).to_excel(writer,sheet_name='new_vars')

Here I get back the rsult into a dict

In [487]:
from pandas import ExcelFile
xls = ExcelFile(input_path+'new_old_correspondance.xlsx')
df = xls.parse(xls.sheet_names[0])
df.set_index('Old',drop=True,inplace=True)
old_new_dict=df['New'].to_dict()

Select only the indicators I'll use:

# TAKE IT FROM HERE

In [488]:
old_df1_target=old_df1.loc[idx[:,:,list(old_new_dict.keys())],:].copy()
new_df1_target=new_df1.loc[(slice(None),slice(None),list(old_new_dict.values())),:].copy()

# Still an issue with the levels
#new_df1_target.index.levels[2]

In [ ]:
# Issue slicing correctly such that I can replace my indexes properly 

#Use set index : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.set_levels.html

In [491]:
old_df1.reset_index()['indic']#.replace(old_new_dict,).unique()

0             105-1.3 OPD Moderate Acute Malnutrition (MAM)
1         105-1.3 OPD Severe Acute Malnutrition With Oedema
2         105-1.3 OPD Severe Acute Malnutrition Without ...
3                        105-2.1 A1:ANC 1st Visit for women
4                        105-2.1 A2:ANC 4th Visit for women
                                ...                        
143355                               EPI - MR 1 doses given
143356                              EPI - PCV 1 doses given
143357                              EPI - PCV 3 doses given
143358    Percentage of babies born with low birth weigh...
143359                 Underweight rate of children under 5
Name: indic, Length: 143360, dtype: object

### SAM and MAM are still an issue, with lack of clarity on what to use : need to reach out to a specialist

In [437]:
print(old_df1.loc[idx[:,:,'105-1.3 OPD Severe Acute Malnutrition Without Oedema'],('with_outliers','2019',['Nov','Dec'])].mean(),
      new_df1.loc[idx[:,:,'105-NA03c1. Identified malnourished clients(<10) this month - SAM using MUAC -  Without Oedema'],('with_outliers','2020',['Jan','Feb'])].mean(),
     new_df1.loc[idx[:,:,'105-NA03d1. Identified malnourished clients(<10) this month - SAM using W/H or L -  Without Oedema'],('with_outliers','2020',['Jan','Feb'])].mean())  

type           year  month
with_outliers  2019  Nov      6.964639
                     Dec      7.057692
dtype: float64 type           year  month
with_outliers  2020  Jan      4.877506
                     Feb      6.157328
dtype: float64 type           year  month
with_outliers  2020  Jan      5.716312
                     Feb      4.076923
dtype: float64


In [436]:
print(old_df1.loc[idx[:,:,'105-1.3 OPD Moderate Acute Malnutrition (MAM)'],('with_outliers','2019',['Nov','Dec'])].mean(),
      new_df1.loc[idx[:,:,'105-NA03a1. Identified malnourished clients(<10) this month - MAM using MUAC'],('with_outliers','2020',['Jan','Feb'])].mean(),
     new_df1.loc[idx[:,:,'105-NA03b1. Identified malnourished clients(<10) this month - MAM using W/H or L'],('with_outliers','2020',['Jan','Feb'])].mean())         

type           year  month
with_outliers  2019  Nov      16.579646
                     Dec      14.520879
dtype: float64 type           year  month
with_outliers  2020  Jan      659.34816
                     Feb      162.30000
dtype: float64 type           year  month
with_outliers  2020  Jan      17.726984
                     Feb      10.997110
dtype: float64


In [477]:
#df1=pd.merge(old_df1,new_df1,left_index=True,right_index=True,how='right')
#df1

In [413]:
len(new_df1)

282791

# Flag outliers

In [286]:
for x in all_months:
    new_df1['without_outliers','2020',x]= np.nan
new_df1

type                                                                    with_outliers  \
year                                                                             2020   
var                                                                               Jan   
district id          indic                                                              
OYAM     ftNNOiF8eLQ 105-AN01a. ANC 1st Visit for women                           NaN   
                     105-AN02. ANC 4th Visit for women                            NaN   
                     105-MA01. Admissions                                         NaN   
                     105-MA04a. Deliveries in unit - Total                        NaN   
                     105-MA04b1. Deliveries in unit -Live births - T...           NaN   
...                                                                               ...   
WAKISO   nYpQROJrK2y EPI - PCV 2 doses_Under 1                                   27.0   
                     EPI - PCV 3 coverage < 1 year (%)                            NaN   
                     EPI - PCV 3 doses_Under 1                                   22.0   
                     EPI - PCV1 Coverage < 1 year (%)                             NaN   
                     NUT: Percentage of children/babies born with lo...           NaN   

type                                                                           \
year                                                                            
var                                                                       Feb   
district id          indic                                                      
OYAM     ftNNOiF8eLQ 105-AN01a. ANC 1st Visit for women                   NaN   
                     105-AN02. ANC 4th Visit for women                    NaN   
                     105-MA01. Admissions                                 NaN   
                     105-MA04a. Deliveries in unit - Total                NaN   
                     105-MA04b1. Deliveries in unit -Live births - T...   NaN   
...                                                                       ...   
WAKISO   nYpQROJrK2y EPI - PCV 2 doses_Under 1                           17.0   
                     EPI - PCV 3 coverage < 1 year (%)                    NaN   
                     EPI - PCV 3 doses_Under 1                           20.0   
                     EPI - PCV1 Coverage < 1 year (%)                     NaN   
                     NUT: Percentage of children/babies born with lo...  50.0   

type                                                                           \
year                                                                            
var                                                                       Mar   
district id          indic                                                      
OYAM     ftNNOiF8eLQ 105-AN01a. ANC 1st Visit for women                   NaN   
                     105-AN02. ANC 4th Visit for women                    NaN   
                     105-MA01. Admissions                                 NaN   
                     105-MA04a. Deliveries in unit - Total                NaN   
                     105-MA04b1. Deliveries in unit -Live births - T...   NaN   
...                                                                       ...   
WAKISO   nYpQROJrK2y EPI - PCV 2 doses_Under 1                           58.0   
                     EPI - PCV 3 coverage < 1 year (%)                    NaN   
                     EPI - PCV 3 doses_Under 1                           62.0   
                     EPI - PCV1 Coverage < 1 year (%)                     NaN   
                     NUT: Percentage of children/babies born with lo...   NaN   

type                                                                           \
year                                                                            
var                                                                   

In [287]:
%%time

cutoff = 2

values = np.zeros(new_df1['with_outliers',].shape)
replacements = 0
names = []
for x in new_df1.index:
    df1 = new_df1.loc[x,('with_outliers',)].values
    if np.nanstd(df1)!=0 and np.isnan(df1).sum()!=len(df1):
        zscore = abs(stats.zscore(df1,nan_policy='omit'))
        data = np.where(zscore>cutoff,np.nanmedian(df1),df1)
        count = np.sum((zscore>cutoff))
        
    else:
        data = df1
        count = 0
    if count > 0:
        names.append(x)
    values[new_df1.index.get_loc(x)]=data
    replacements = replacements + count
    

new_df1.loc[:,('without_outliers',)]=values
print("\n",str(replacements)," replacements made")
new_df1.loc[names,:]

C:\Users\Dalberg\anaconda3\envs\icohs\lib\site-packages\numpy\lib\nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
<timed exec>:10: RuntimeWarning: invalid value encountered in greater
<timed exec>:11: RuntimeWarning: invalid value encountered in greater
C:\Users\Dalberg\anaconda3\envs\icohs\lib\site-packages\IPython\core\interactiveshell.py:3331: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)



 0  replacements made
Wall time: 4min 54s


Empty DataFrame
Columns: [(with_outliers, 2020, Jan), (with_outliers, 2020, Feb), (with_outliers, 2020, Mar), (with_outliers, 2020, Apr), (with_outliers, 2020, May), (without_outliers, 2020, Jan), (without_outliers, 2020, Feb), (without_outliers, 2020, Mar), (without_outliers, 2020, Apr), (without_outliers, 2020, May)]
Index: []

In [341]:
#Test
#new_df1.loc[idx["ABIM",:,'105-AN01a. ANC 1st Visit for women'],:]

In [304]:
new_df_all=new_df.copy().iloc[:,0].reset_index().groupby(['district','id']).sum().groupby('district').count()
report_tot=new_df_all.values

In [332]:
new_df1_report=new_df1['with_outliers',].copy().reset_index().groupby(['district','indic']).count()#/report_tot
for x in all_months:
    new_df1_report['reporting_perc',x]= np.nan
    new_df1_report['reporting_perc',x]=new_df1_report['2020',x]/new_df1_report['id',]
new_df1_report

C:\Users\Dalberg\anaconda3\envs\icohs\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
C:\Users\Dalberg\anaconda3\envs\icohs\lib\site-packages\IPython\core\interactiveshell.py:2857: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(


year                                                         id 2020          \
var                                                              Jan Feb Mar   
district indic                                                                 
ABIM     105-AN01a. ANC 1st Visit for women                  21   18  18  18   
         105-AN02. ANC 4th Visit for women                   21   18  18  17   
         105-MA01. Admissions                                21   18  18  18   
         105-MA04a. Deliveries in unit - Total               21   18  18  18   
         105-MA04b1. Deliveries in unit -Live births - T...  21   17  18  18   
...                                                          ..  ...  ..  ..   
ZOMBO    EPI - PCV 2 doses_Under 1                           22   16  16  15   
         EPI - PCV 3 coverage < 1 year (%)                   22    0   0   0   
         EPI - PCV 3 doses_Under 1                           22   16  16  14   
         EPI - PCV1 Coverage < 1 year (%)                    22    0   0   0   
         NUT: Percentage of children/babies born with lo...  22   11  11   8   

year                                                                 \
var                                                         Apr May   
district indic                                                        
ABIM     105-AN01a. ANC 1st Visit for women                  18   0   
         105-AN02. ANC 4th Visit for women                   17   0   
         105-MA01. Admissions                                18   0   
         105-MA04a. Deliveries in unit - Total               18   0   
         105-MA04b1. Deliveries in unit -Live births - T...  17   0   
...                                                          ..  ..   
ZOMBO    EPI - PCV 2 doses_Under 1                           16   0   
         EPI - PCV 3 coverage < 1 year (%)                    0   0   
         EPI - PCV 3 doses_Under 1                           16   0   
         EPI - PCV1 Coverage < 1 year (%)                     0   0   
         NUT: Percentage of children/babies born with lo...  11   0   

year                                                        reporting_perc  \
var                                                                    Jan   
district indic                                                               
ABIM     105-AN01a. ANC 1st Visit for women                       0.857143   
         105-AN02. ANC 4th Visit for women                        0.857143   
         105-MA01. Admissions                                     0.857143   
         105-MA04a. Deliveries in unit - Total                    0.857143   
         105-MA04b1. Deliveries in unit -Live births - T...       0.809524   
...                                                                    ...   
ZOMBO    EPI - PCV 2 doses_Under 1                                0.727273   
         EPI - PCV 3 coverage < 1 year (%)                        0.000000   
         EPI - PCV 3 doses_Under 1                                0.727273   
         EPI - PCV1 Coverage < 1 year (%)                         0.000000   
         NUT: Percentage of children/babies born with lo...       0.500000   

year                                                                   \
var                                                               Feb   
district indic                                                          
ABIM     105-AN01a. ANC 1st Visit for women                  0.857143   
         105-AN02. ANC 4th Visit for women                   0.857143   
         105-MA01. Admissions                                0.857143   
         105-MA04a. Deliveries in unit - Total               0.857143   
         105-MA04b1. Deliveries in unit -Live births - T...  0.857143   
...                                                               ...   
ZOMBO    EPI - PCV 2 doses_Under 1                           0.727273   
         EPI - PCV 3 coverage < 1 year (%)                   

# Tests and Issues to address:

Possibility of outliers in data rare lines (E.g. 2 data points only, one right, one completely off. WOuld need an alternative layer for those
- might run an added test this time based on columns values
- or could just exclude any facility that has reported less than twice (not great, would delete a lot of useful data)

eg. look at xuCJ8NBomxP	Gombe (Butambala) Hospital 'NUT: Percentage of children/babies born with low birth weight (<2.5kg)' outlier in April 2020 > here it is a %, not a absolute number
Would alternatively only use % and population normalized metrics?

In [244]:
e=pd.Series([8.2,5.3,3.9,93.6])
zscore = stats.zscore(e.fillna(e.mean()))
#flags = abs(zscore) > 1.9
#flags == True
np.where(zscore>cutoff,np.nanmedian(e),e)
np.sum((zscore>1))

1

# Look at SAM data

In [ ]:
lbw_indic=['NUT: Percentage of children/babies born with low birth weight (<2.5kg)']

In [ ]:
lbw_new=pd.read_csv(input_path+'lbw_facility_pivot.csv')
lbw_new=get_clean_pivot(lbw_new)

In [ ]:
dropped_months=['Nov','Dec']
lbw_new.drop(dropped_months,axis=1,inplace=True,level=2)

# Look at all facilities

In [ ]:
lbw_rep_all=lbw_new.copy().groupby('district').count()

In [ ]:
lbw_rep_all.sum()

In [ ]:
# Should buid a function out of that 

for m in ['Jan','Feb','Mar','Apr']:
    lbw_rep_all[('2020','lbw reporting',m)]=lbw_rep_all[('2020','NUT: Percentage of children/babies born with low birth weight (<2.5kg)',m)]/lbw_rep_all['facility']

In [ ]:
lbw_rep_all.sum(axis=0)/135

# Look at Gvt facilites only

In [ ]:
facility = pd.read_csv(facility_path+'MFL-Dec-19.csv')

In [ ]:
facility['index_code']=facility['District'].apply(lambda x: x[:-9].upper())+facility['Health Facility']
facility.set_index('index_code',drop=True,inplace=True)

In [ ]:
facility_gov=facility[facility['Ownership']=='GOV'].copy()

In [ ]:
len(list(set(facility_gov.index).difference(set(lbw_new.index))))


In [ ]:
lbw_new['index_code']=lbw_new['district']+lbw_new['facility']
lbw_new.set_index('index_code',drop=False,inplace=True)

In [ ]:
lbw_gvt=pd.merge(facility_gov,lbw_new,how='left',left_index=True,right_index=True)

Paused here

In [ ]:
lbw_gvt
#multi_index=pd.MultiIndex.from_tuples([('Test','test','test')])
#data_elements=pd.DataFrame(index=districts,columns=multi_index)

In [ ]:
lbw_rep_gvt=lbw_gvt.copy().groupby('district').count()

In [ ]:
# Should buid a function out of that 

for m in ['Jan','Feb','Mar','Apr']:
    lbw_rep_all[('2020','lbw reporting',m)]=lbw_rep_all[('2020','NUT: Percentage of children/babies born with low birth weight (<2.5kg)',m)]/lbw_rep_all['facility']

In [ ]:
lbw_rep_all.sum(axis=0)/135

## Get that to csv

In [ ]:
sam_export_df=sam_df.copy()
sam_export_df.columns=sam_df.columns.map("_".join)
sam_export_df.to_csv(output_path+'sam_map.csv')

Note that I am not looking at old data here, because I coul not find teh corresponding indicators execpt for two of them